In [8]:
import os
import pandas as pd
import geopandas as gp
import rasterio

In [2]:
NB_USER = os.getenv('NB_USER')
print(NB_USER)

jovyan


In [3]:
aoi_path = os.getenv('AOI')
aoi_path

'/home/jovyan/work/notebooks/aoi.geojson'

In [6]:
aoi = gp.read_file(aoi_path)

In [7]:
aoi

,geometry
0,"POLYGON ((35.46387 50.43652, 34.66187 50.02892..."


In [ ]:
def create_ndvi(self, window_size_pixels=5120):
    np.seterr(divide='ignore', invalid='ignore')
    with rasterio.open(self.raster_list['rasters']['red']['path'], 'r') as red_ds:
        with rasterio.open(self.raster_list['rasters']['nir']['path'], 'r') as nir_ds:
            if red_ds.shape != nir_ds.shape:
                raise ValueError("Shape of 'red'and 'nir' files nonequal. "
                                 "They shoud have equal height and width ")
            ndvi_path = os.path.join(os.path.dirname(self.raster_list['rasters']['red']['path']),
                                     'project_index_ndvi.tif')
            metadata, bounds, xres, yres = self.rasterMetaData_masked(red_ds.profile)
            ndvi_meta = metadata.copy()
            ndvi_meta.update({"dtype": "float32",
                              "nodata": -10000,
                              "count": 1})
            with rasterio.open(ndvi_path, 'w+', BIGTIFF='IF_NEEDED', **ndvi_meta) as output_file:
                # if not window_size_pixels:
                #     window_size_pixels = np.ceil(window_size / xres)
                window_config = tiles_with_overlap_shape(red_ds.meta['height'], red_ds.meta['width'],
                                                         window_size_pixels, 1)
                window_config = np.array(window_config)
                windows_coordinates = [pt for pt in window_config]
                progress_bar = tqdm()
                for window in windows_coordinates:
                    progress_bar.set_postfix(Row='{}'.format(window[0]), Col='{}'.format(window[2]))
                    img_window = ((int(window[0]), int(window[1])), (int(window[2]), int(window[3])))
                    res_red = red_ds.read(window=img_window).astype(np.float32)
                    res_nir = nir_ds.read(window=img_window).astype(np.float32)
                    if res_nir.max() > 256:
                        logging.error("Raster is not scaled properly max_value- {}".format(res_nir.max()))
                        res_nir[res_nir < 0] = 0
                        res_red[res_red < 0] = 0
                        res_nir = self.raster_values_normalization(res_nir, image_array_min=0,
                                                                   image_array_max=65536)
                        res_red = self.raster_values_normalization(res_red, image_array_min=0,
                                                                   image_array_max=65536)
                    window_raster = ((res_nir[0] - res_red[0]) / (res_nir[0] + res_red[0])).astype(np.float32)
                    # masking nans
                    nan_mask = np.isnan(window_raster)
                    window_raster[nan_mask] = -10000
                    output_file.write(window_raster, window=img_window, indexes=1)
                    progress_bar.update(1)
    ndvi_descr = {'file_name': ndvi_path.split("/")[-1],
                  'path': ndvi_path}
    return ndvi_descr, ndvi_meta

In [ ]:
def rasterMetaData_masked(self, profile):
    transform = profile['transform']
    blockxsize = profile.get('blockxsize', 512)
    blockysize = profile.get('blockysize', 512)
    tiled = profile.get('tiled', False) or self.tiled
    width = profile['width']
    height = profile['height']
    crs = profile['crs']
    bounds = namedtuple('bounds', ['left', 'bottom', 'right', 'top'])
    bounds.left = profile['transform'][2]
    bounds.top = profile['transform'][5]
    bounds.right, bounds.bottom = (profile['transform'] * [profile['width'], profile['height']])
    # bounds = meta['bounds']
    profile = {
        'count': 1,
        'crs': crs,
        'dtype': rasterio.float32,
        'driver': "GTiff",
        'height': height,
        'width': width,
        'nodata': -10000,
        'transform': transform,
        'blockxsize': blockxsize,
        'blockysize': blockysize,
        'tiled': tiled
    }
    xres = transform.a
    yres = abs(transform.e)
    return profile, bounds, xres, yres

In [9]:
rasterio.open?

Signature: rasterio.open(fp, mode='r', driver=None, width=None, height=None, count=None, crs=None, transform=None, dtype=None, nodata=None, sharing=False, **kwargs)
Docstring:
Open a dataset for reading or writing.

The dataset may be located in a local file, in a resource located by
a URL, or contained within a stream of bytes.

In read ('r') or read/write ('r+') mode, no keyword arguments are
required: these attributes are supplied by the opened dataset.

In write ('w' or 'w+') mode, the driver, width, height, count, and dtype
keywords are strictly required.

Parameters
----------
fp : str, file object or pathlib.Path object
    A filename or URL, a file object opened in binary ('rb') mode,
    or a Path object.
mode : str, optional
    'r' (read, the default), 'r+' (read/write), 'w' (write), or
    'w+' (write/read).
driver : str, optional
    A short format driver name (e.g. "GTiff" or "JPEG") or a list of
    such names (see GDAL docs at
    http://www.gdal.org/formats_list.html).